# Stock Price Recommender System

- PyTorch (Not TF)
- yfinance (for now just use NASDAQ directly)
- matplotlib
- numpy
- pandas
- plan to use h2o.ai wave

**Contributor**: James Guzman

**Teammates**: Josef, Anshul

**Project Advisor**: Dr. Magdalini Eirinaki

**SJSU CMPE 256**: Advanced Data Mining

**Date**: Nov 22, 2023

Based on "Stock Recommendation System with Python/SQL":

- https://youtube.com/playlist?list=PL9ATnizYJ7f9sW_mdY3071dS8mMwBGu2S&si=S9YpLhQsMvFr3AFn

Other Resources:

- Create Finance Database with Py & SQL
- Use MACD indicator and RSI/SMA indicator for decision makers

NOTE: For our Stock Recommender System application, it may make sense to build up the data prep pipelines and DL deployment pipelines in NiFi and provide a GUI by H2O.ai wave to make it easier
for our AI/DL audience to see our data pipeline. We still have to create the custom NiFi python
processors for this task.

In [1]:
import os
import ta
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from copy import deepcopy as dc
from tqdm.notebook import tqdm
# import yfinance as yf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
# import streamlit
    # streamlit they use to deploy as a web app, for now I will use Wave or PySide6

## Ingest Stock Close Price Predicted Data

We can read in our GOOG closing stock price predictions table that also has the Date of the prediction like Algovibes does, but our stock recommender system is based on using our PyTorch predictions when calculating **MACDecision**, etc.

In [2]:
stock_dir_path = "/home/james/src/datasets/NASDAQ_Yahoo_Finance_PyTorch/stocks"

In [3]:
symbol_name = "GOOG"

In [4]:
close_price_pred_filename = f"pytorch_final_closing_price_preds_{symbol_name}.csv"

In [5]:
google_stock_price_pred_pd = pd.read_csv(os.path.join(stock_dir_path, close_price_pred_filename))

In [6]:
google_stock_price_pred_pd.head()

,100th Date,Close
0,2017-03-16,780.899261
1,2017-03-17,781.577956
2,2017-03-20,782.921275
3,2017-03-21,779.192227
4,2017-03-22,775.225077


## Compute Technical Stock Indicators

First we will compute **MACDecision** stock indicator using our PyTorch predicted stock closing prices.
We then create the **RSIDecision Stock Indicator** function.

In [10]:
def MACDecision(stock_price_pred_df):
    stock_price_pred_df["MACD_diff"] = ta.trend.macd_diff(stock_price_pred_df.Close)
    stock_price_pred_df.loc[(stock_price_pred_df.MACD_diff > 0) & 
                            (stock_price_pred_df.MACD_diff.shift(1) < 0), "Decision MACD"] = "Buy"

**NOTE**: if we use SMA 200, we should most likely have a our PyTorch model be trained on predicting
final closing stock price based on learning from closing stock prices from 200 days sliding window
Else, we can try SMA 100 since we trained our PyTorch model based on 100 days sliding window of
past closing prices to predict final closing stock price

In [11]:
def RSI_SMA_Decision(stock_price_pred_df):
    # NOTE: our PyTorch model was trained on 100 days to predict final closing price
    stock_price_pred_df["RSI"] = ta.momentum.rsi(stock_price_pred_df.Close, window=10)
    # This probably would be simple moving average based on 200 days of predicted closing stock prices
    stock_price_pred_df["SMA200"] = ta.trend.sma_indicator(stock_price_pred_df.Close, window = 200)
    stock_price_pred_df.loc[(stock_price_pred_df.Close > stock_price_pred_df.SMA200) &
                            (stock_price_pred_df.RSI < 30), "Decision RSI/SMA"] = "Buy"

Apply MACDecision indicator and RSIDecision indicator to GOOG predicted closing stock prices dataframe. Can be applied to all other company dataframes where previously we predicted closing stock prices.

In [12]:
# NOTE: this single element list will be replaced by a list of company stock predicted close price
# dataframes
# for stock_price_pred_df in [google_stock_price_pred_pd]:
    # Compute Technical Stock Indicator for each company's stock closing predicted price
MACDecision(google_stock_price_pred_pd)
RSI_SMA_Decision(google_stock_price_pred_pd)

In [14]:
google_stock_price_pred_pd

,100th Date,Close,MACD_diff,Decision MACD,RSI,SMA200,Decision RSI/SMA
0,2017-03-16,780.899261,NaN,NaN,NaN,NaN,NaN
1,2017-03-17,781.577956,NaN,NaN,NaN,NaN,NaN
2,2017-03-20,782.921275,NaN,NaN,NaN,NaN,NaN
3,2017-03-21,779.192227,NaN,NaN,NaN,NaN,NaN
4,2017-03-22,775.225077,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
762,2020-03-26,1060.627488,0.839057,NaN,73.891365,1010.041608,NaN
763,2020-03-27,1064.726344,0.881474,NaN,78.147480,1010.525653,NaN
764,2020-03-30,1068.556381,1.044949,NaN,81.310642,1011.027595,NaN
765,2020-03-31,1069.329974,1.078294,NaN,81.898669,1011.557901,NaN


## Buy Asset On Next Day?

From our results table based on stock closing price prediction, **Decision MACD** and/or **Decision RSI/SMA** will indicate if there is a buy signal for our predictions.

Next we'll check the very last row.

In [15]:
if google_stock_price_pred_pd["Decision MACD"].iloc[-1] == "Buy":
    print("Buying Signal MACD for GOOG")
if google_stock_price_pred_pd["Decision RSI/SMA"].iloc[-1] == "Buy":
    print("Buying Signal RSI/SMA for GOOG")